In [2]:
with open('api-key.txt', 'r') as f:
    lines = f.readlines()
    apikey = lines[0]

In [21]:
import requests
import pandas as pd
from io import StringIO

api_url = "https://api.covalenthq.com/v1/{chain_id}/events/topics/{topic_id}/"
chain_id = 30
topic_id = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
req_url = api_url.format(chain_id = chain_id, topic_id = topic_id)

p_start_block = 3397795
p_end_block = 3478615
p_page_size = 1000
p_currency = "usd"
p_format = "csv"
p_key = apikey
q_params = { 
    "starting-block" : p_start_block
    , "ending-block" : p_end_block
    , "page-size" : p_page_size
    , "quote-currency" : p_currency
    , "format" : p_format
    , "key" : p_key
}


In [22]:
df_lst = list()
q_params["starting-block"] = p_start_block
print("Starting download")
while (q_params["starting-block"] <= q_params["ending-block"]):
    resp = requests.get(req_url, params = q_params)
    try:
        t_df = pd.read_csv(StringIO(resp.text), sep = ",")
        print("Success retrieval from block {}".format(q_params["starting-block"]))
        new_start = t_df["block_height"].max()
        q_params["starting-block"] = new_start + 1
        df_lst.append(t_df)
        
    except Exception as err:
        exception_type = type(err).__name__
        print("{}:{}".format(exception_type,err))
        print("Ending block {}".format(q_params["starting-block"]))
        q_params["starting-block"] = q_params["ending-block"] + 1
    

Starting download
Success retrieval from block 3397795
Success retrieval from block 3398588
Success retrieval from block 3399252
Success retrieval from block 3399823
Success retrieval from block 3400787
Success retrieval from block 3401772
Success retrieval from block 3402230
Success retrieval from block 3402548
Success retrieval from block 3403065
Success retrieval from block 3403928
Success retrieval from block 3404698
Success retrieval from block 3405222
Success retrieval from block 3406583
Success retrieval from block 3407245
Success retrieval from block 3407705
Success retrieval from block 3407971
Success retrieval from block 3408401
Success retrieval from block 3409412
Success retrieval from block 3410361
Success retrieval from block 3411194
Success retrieval from block 3412814
Success retrieval from block 3413849
Success retrieval from block 3414709
Success retrieval from block 3415675
Success retrieval from block 3415908
Success retrieval from block 3416228
Success retrieval fr

In [23]:
#filter transfers from only, and select only the needed columns
def shape_it(in_df):
    n_df = in_df[in_df["decoded_params_name"] == "from"] \
        .filter(items=["block_signed_at", "block_height", "tx_hash", "sender_contract_ticker_symbol", "decoded_params_name", "decoded_params_type", "decoded_params_value"])
    return n_df

In [24]:
f_df = pd.concat(map(shape_it, df_lst))

In [25]:
f_df.head()

,block_signed_at,block_height,tx_hash,sender_contract_ticker_symbol,decoded_params_name,decoded_params_type,decoded_params_value
0,2021-06-01T00:01:41Z,3397799,0x8670cc25a6c54daa9d9a2dc9772e817b2b0c466e048f...,WRBTC,from,address,0xa917bf723433d020a15629eba71f6c2a6b38e52d
4,2021-06-01T00:01:41Z,3397799,0x8670cc25a6c54daa9d9a2dc9772e817b2b0c466e048f...,DOC,from,address,0xd715192612f03d20bae53a5054af530c9bb0fa3f
8,2021-06-01T00:02:25Z,3397800,0x0d548e439850983e21fb1774449333c94231e174783e...,ETHes,from,address,0x4bf113905d7f69202106f613308bb02c84aadf2f
12,2021-06-01T00:02:25Z,3397800,0x0d548e439850983e21fb1774449333c94231e174783e...,ETHes,from,address,0x1ccad820b6d031b41c54f1f3da11c0d48b399581
16,2021-06-01T00:02:25Z,3397800,0x0d548e439850983e21fb1774449333c94231e174783e...,ETHes,from,address,0x1ccad820b6d031b41c54f1f3da11c0d48b399581


In [26]:
#Add date
f_df["block_signed_date"] = f_df.apply(lambda p: pd.Timestamp(p["block_signed_at"]).date() , axis = 1)

In [27]:
f_df.head()

,block_signed_at,block_height,tx_hash,sender_contract_ticker_symbol,decoded_params_name,decoded_params_type,decoded_params_value,block_signed_date
0,2021-06-01T00:01:41Z,3397799,0x8670cc25a6c54daa9d9a2dc9772e817b2b0c466e048f...,WRBTC,from,address,0xa917bf723433d020a15629eba71f6c2a6b38e52d,2021-06-01
4,2021-06-01T00:01:41Z,3397799,0x8670cc25a6c54daa9d9a2dc9772e817b2b0c466e048f...,DOC,from,address,0xd715192612f03d20bae53a5054af530c9bb0fa3f,2021-06-01
8,2021-06-01T00:02:25Z,3397800,0x0d548e439850983e21fb1774449333c94231e174783e...,ETHes,from,address,0x4bf113905d7f69202106f613308bb02c84aadf2f,2021-06-01
12,2021-06-01T00:02:25Z,3397800,0x0d548e439850983e21fb1774449333c94231e174783e...,ETHes,from,address,0x1ccad820b6d031b41c54f1f3da11c0d48b399581,2021-06-01
16,2021-06-01T00:02:25Z,3397800,0x0d548e439850983e21fb1774449333c94231e174783e...,ETHes,from,address,0x1ccad820b6d031b41c54f1f3da11c0d48b399581,2021-06-01


In [28]:
#select the required fields, and get unique wallets for each day
uf_df = f_df.filter(items = ["block_signed_date", "decoded_params_value"]).drop_duplicates()

In [29]:
uf_df.head()

,block_signed_date,decoded_params_value
0,2021-06-01,0xa917bf723433d020a15629eba71f6c2a6b38e52d
4,2021-06-01,0xd715192612f03d20bae53a5054af530c9bb0fa3f
8,2021-06-01,0x4bf113905d7f69202106f613308bb02c84aadf2f
12,2021-06-01,0x1ccad820b6d031b41c54f1f3da11c0d48b399581
20,2021-06-01,0xc6aaa6ee64339008d4c3ff8820f886116b869f9f


In [35]:
import altair as alt

alt.data_transformers.enable('vegafusion') #enables plotting more data point (>5k)

DataTransformerRegistry.enable('vegafusion')

In [36]:
uf_df["block_signed_date"] = pd.to_datetime(uf_df["block_signed_date"])

In [40]:
#alt.Chart(uf_df, width = 1920, height = 1080) \
alt.Chart(uf_df, width = 1024, height = 768) \
    .mark_bar(color = "#00D8D5") \
    .configure(
        background = 'black'
        ) \
    .configure_title(
        anchor='start'
        , color='white'
        , fontSize = 80
        , font = 'Titillium Web'
        ) \
    .configure_axis(
        gridWidth = 0
        , labelColor = 'white'
        , labelFont = 'Titillium Web'
        , labelFontSize = 50
        , titleFont = 'Titillium Web'
        , titleFontSize = 60
        , titleColor = 'white'
        ) \
    .encode(
        alt.X(field ='block_signed_date', type = 'temporal', timeUnit='date', title = 'Date'),
        alt.Y(field = 'decoded_params_value', type = 'ordinal', aggregate = 'count', title="Number of wallets")
        ) \
    .properties(
        title='RSK peaks just under 1000 Daily Active Users in June 2021' 
        )
    

alt.Chart(...)

In [39]:
# Number of transfers on RSK
# change chart size 1920x1080
# add footer
# add watermark
# increase font size
# Left aligned title
# 

#alt.Chart(f_df, width = 1920, height = 1080) \
alt.Chart(f_df, width = 1024, height = 768) \
    .mark_bar(color = "#00D8D5") \
    .configure(
        background = 'black'
        ) \
    .configure_title(
        anchor='start'
        , color='white'
        , fontSize = 80
        , font = 'Titillium Web'
        ) \
    .configure_axis(
        gridWidth = 0
        , labelColor = 'white'
        , labelFont = 'Titillium Web'
        , labelFontSize = 50
        , titleFont = 'Titillium Web'
        , titleFontSize = 60
        , titleColor = 'white'
        ) \
    .encode(
        alt.X(field ='block_signed_date', type = 'temporal', timeUnit='date', title = 'Date'),
        alt.Y(field = 'decoded_params_value', type = 'ordinal', aggregate = 'count', title="Number of Transfers")
        ) \
    .properties(
        title='Transfers on RSK Nears 15k in June 2021'
        )
        

alt.Chart(...)